<a href="https://colab.research.google.com/github/ravi-gopalan/DAND_Data_Wrangling/blob/master/wikidata_sparql_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sparqlwrapper

# https://rdflib.github.io/sparqlwrapper/

     |████████████████████████████████| 348kB 22.7MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 


In [0]:
import pandas as pd
import json
from SPARQLWrapper import SPARQLWrapper, JSON


def get_results(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
      item = []
      for c in cols:
        item.append(row.get(c, {}).get('value'))
      out.append(item)
    return pd.DataFrame(out, columns = cols)

In [0]:
endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?noodle ?noodleLabel ?country_of_origin ?country_of_originLabel ?instance_of ?instance_ofLabel ?subclass_of ?subclass_ofLabel  ?Commons_category 
WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "   [AUTO_LANGUAGE],en". }
  ?noodle wdt:P279 wd:Q192874.
  OPTIONAL { ?noodle wdt:P495 ?country_of_origin. }
  OPTIONAL { ?noodle wdt:P31 ?instance_of. }
  OPTIONAL { ?noodle wdt:P279 ?subclass_of. }
  OPTIONAL { ?noodle wdt:P373 ?Commons_category. }  
}"""


df = get_results(endpoint_url, query)

In [11]:
df.head()

,noodle,noodleLabel,country_of_origin,country_of_originLabel,instance_of,instance_ofLabel,subclass_of,subclass_ofLabel,Commons_category
0,http://www.wikidata.org/entity/Q20065,Spätzle,http://www.wikidata.org/entity/Q142,France,None,None,http://www.wikidata.org/entity/Q192874,noodle,Spaetzle
1,http://www.wikidata.org/entity/Q20065,Spätzle,http://www.wikidata.org/entity/Q183,Germany,None,None,http://www.wikidata.org/entity/Q192874,noodle,Spaetzle
2,http://www.wikidata.org/entity/Q20065,Spätzle,http://www.wikidata.org/entity/Q142,France,None,None,http://www.wikidata.org/entity/Q53619707,egg pasta,Spaetzle
3,http://www.wikidata.org/entity/Q20065,Spätzle,http://www.wikidata.org/entity/Q183,Germany,None,None,http://www.wikidata.org/entity/Q53619707,egg pasta,Spaetzle
4,http://www.wikidata.org/entity/Q34156,Korean noodles,http://www.wikidata.org/entity/Q18097,Korea,None,None,http://www.wikidata.org/entity/Q192874,noodle,Noodles from Korea


In [12]:
df[['noodleLabel','country_of_originLabel','instance_ofLabel','subclass_ofLabel','Commons_category']]

,noodleLabel,country_of_originLabel,instance_ofLabel,subclass_ofLabel,Commons_category
0,Spätzle,France,None,noodle,Spaetzle
1,Spätzle,Germany,None,noodle,Spaetzle
2,Spätzle,France,None,egg pasta,Spaetzle
3,Spätzle,Germany,None,egg pasta,Spaetzle
4,Korean noodles,Korea,None,noodle,Noodles from Korea
...,...,...,...,...,...
76,extruded noodle,None,None,noodle,Extruded noodles
77,knife-cut noodle,None,None,noodle,Knife-cut noodles
78,Mì,Vietnam,None,noodle,None
79,ribbon noodle,None,type of pasta,pasta,None


In [13]:
df[['noodleLabel','country_of_originLabel','instance_ofLabel','subclass_ofLabel']]['noodleLabel'].unique()

array(['Spätzle', 'Korean noodles', 'zhajiangmian', 'ramen',
       'Curry noodle', 'udon', 'sōmen', 'soba', 'hot dry noodles',
       'cart noodle', 'cellophane noodles', 'jajangmyeon', 'Bakmi',
       'rice noodles', 'Hokkien mee', 'fried noodles', 'Pancit',
       'Chinese noodles', 'Wonton noodles', 'buckwheat noodles', 'kesme',
       'black noodles', 'Bánh hỏi', 'Char kway teow',
       'Chinkiang pot cover noodles', 'Mi rebus', 'dragon beard noodles',
       'phat si io', 'Vietnamese noodles', 'kishimen', 'Kadaif noodles',
       'cold noodle', 'Buldak Bokkeum Myun', 'Mie ayam',
       'Japanese noodles', 'boat noodles', 'Q24933621',
       'Singapore chow mein', 'Q27566431', 'narrow lapsha', 'wide lapsha',
       'laghman', 'extruded noodle', 'knife-cut noodle', 'Mì',
       'ribbon noodle'], dtype=object)

In [15]:
df[['noodleLabel','country_of_originLabel','instance_ofLabel','subclass_ofLabel','Commons_category']]['Commons_category'].unique()

array(['Spaetzle', 'Noodles from Korea', 'Zha jiang mian', 'Ramen', None,
       'Udon', 'Somen', 'Soba', 'Re gan mian', 'Cart noodle',
       'Cellophane noodles', 'Jajangmyeon', 'Noodles from Indonesia',
       'Rice noodle', 'Hokkien mee', 'Fried noodles', 'Pancit',
       'Noodles from China', 'Wonton noodle', 'Kesme', 'Bánh hỏi',
       'Char kway teow', 'Bakmi Jawa', 'Phat si-io',
       'Noodles from Vietnam', 'Kishimen', 'Tel kadayıf', 'Cold noodles',
       'Mie ayam', 'Noodles from Japan', 'Laghman', 'Extruded noodles',
       'Knife-cut noodles'], dtype=object)

In [16]:
df.groupby(['Commons_category','noodleLabel'])['noodleLabel'].agg('count')

Commons_category        noodleLabel       
Bakmi Jawa              Mi rebus              1
Bánh hỏi                Bánh hỏi              1
Cart noodle             cart noodle           2
Cellophane noodles      cellophane noodles    2
Char kway teow          Char kway teow        1
Cold noodles            cold noodle           1
Extruded noodles        extruded noodle       1
Fried noodles           fried noodles         2
Hokkien mee             Hokkien mee           2
Jajangmyeon             jajangmyeon           3
Kesme                   kesme                 2
Kishimen                kishimen              2
Knife-cut noodles       knife-cut noodle      1
Laghman                 laghman               2
Mie ayam                Mie ayam              3
Noodles from China      Chinese noodles       1
Noodles from Indonesia  Bakmi                 2
Noodles from Japan      Japanese noodles      1
Noodles from Korea      Korean noodles        1
Noodles from Vietnam    Vietnamese noodles   